In [12]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [34]:
import pandas as pd
import featuretools as ft
import numpy as np

df = pd.read_csv('~/Desktop/UNEP-STARTHACK22/Dataset/Test_michi.csv') # train
df.head()

,Sample_ID,elevation_dif,elevation_var,slope_var,sdoif_var,elevation,slope,placurv,procurv,sdoif,aspect,geology,elevation_bins,rel_sdoif,ring1_elevation,ring2_elevation,weathered,slope_x_twi
0,10865,14,22.750000,184.762308,5.493354e-08,117.00,28.226128,-0.010110,0.002710,1.310381,5.0,2,2,0.006415,3.750,4.6875,1,113.717000
1,10866,8,5.073333,22.164854,3.025842e-08,184.64,19.058626,-0.021890,-0.019203,1.333102,8.0,2,3,0.029680,1.250,2.3750,1,116.714699
2,10867,8,4.660000,108.152137,8.120009e-08,37.92,17.829861,-0.009192,-0.029864,1.238629,12.0,2,3,-0.064793,1.000,2.0625,1,76.263451
3,10868,17,18.856667,2.403042,2.697027e-09,132.24,30.864442,-0.013919,-0.000479,1.300848,8.0,3,1,-0.004564,2.250,4.2500,1,144.897876
4,10869,16,27.740000,8.044256,2.146904e-08,332.64,35.562708,0.013876,-0.002703,1.355567,5.0,3,2,0.051601,2.625,5.5625,1,125.528925


In [35]:
es = ft.EntitySet(id='es')

In [36]:
from woodwork.logical_types import Categorical

es = es.add_dataframe(dataframe_name='df', 
                      dataframe=df, 
                      index = 'Sample_ID')

In [37]:
es

Entityset: es
  DataFrames:
    df [Rows: 5430, Columns: 18]
  Relationships:
    No relationships

In [38]:
features, feature_names = ft.dfs(entityset=es, target_dataframe_name='df', 
                                 trans_primitives = [
                                                     'add_numeric', 
                                                     'multiply_numeric', 
                                                     'diff',
                                                     'square_root',
                                                     'natural_logarithm',
                                                     'rolling_min',
                                                     'rolling_mean',
                                                     'cum_sum',
                                                    'greater_than',
                                                    'rolling_count',
                                                     'rolling_max',
                                                     'absolute'])

/opt/anaconda3/lib/python3.9/site-packages/featuretools/synthesis/deep_feature_synthesis.py:156: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn("Only one dataframe in entityset, changing max_depth to "
/opt/anaconda3/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:312: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['rolling_count', 'rolling_max', 'rolling_mean', 'rolling_min']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py

In [39]:
features.shape

(5430, 646)

In [40]:
features.replace([np.inf, -np.inf], np.nan, inplace=True)

In [41]:
features = features.dropna(axis=1)

In [42]:
features.isnull().sum().any()

False

In [43]:
features.reset_index(inplace=True)
features

/var/folders/32/kbrblf_n7wlc1ts4vm4zwj1w0000gn/T/ipykernel_1699/1417444288.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  features.reset_index(inplace=True)


,Sample_ID,elevation_dif,elevation_var,slope_var,sdoif_var,elevation,slope,placurv,procurv,sdoif,...,SQUARE_ROOT(elevation_var),SQUARE_ROOT(geology),SQUARE_ROOT(ring1_elevation),SQUARE_ROOT(ring2_elevation),SQUARE_ROOT(sdoif),SQUARE_ROOT(sdoif_var),SQUARE_ROOT(slope),SQUARE_ROOT(slope_var),SQUARE_ROOT(slope_x_twi),SQUARE_ROOT(weathered)
0,10865,14,22.750000,184.762308,5.493354e-08,117.00,28.226128,-0.010110,0.002710,1.310381,...,4.769696,1.414214,1.936492,2.165064,1.144719,0.000234,5.312827,13.592730,10.663817,1.0
1,10866,8,5.073333,22.164854,3.025842e-08,184.64,19.058626,-0.021890,-0.019203,1.333102,...,2.252406,1.414214,1.118034,1.541104,1.154600,0.000174,4.365619,4.707956,10.803458,1.0
2,10867,8,4.660000,108.152137,8.120009e-08,37.92,17.829861,-0.009192,-0.029864,1.238629,...,2.158703,1.414214,1.000000,1.436141,1.112937,0.000285,4.222542,10.399622,8.732895,1.0
3,10868,17,18.856667,2.403042,2.697027e-09,132.24,30.864442,-0.013919,-0.000479,1.300848,...,4.342426,1.732051,1.500000,2.061553,1.140547,0.000052,5.555578,1.550175,12.037353,1.0
4,10869,16,27.740000,8.044256,2.146904e-08,332.64,35.562708,0.013876,-0.002703,1.355567,...,5.266878,1.732051,1.620185,2.358495,1.164288,0.000147,5.963448,2.836240,11.203969,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5425,16290,5,1.906667,14.129515,2.751173e-09,29.36,12.414156,0.005201,0.008673,1.309199,...,1.380821,2.236068,0.866025,1.224745,1.144202,0.000052,3.523373,3.758925,7.689109,0.0
5426,16291,13,14.843333,3.479386,2.019093e-09,148.48,29.365934,-0.031899,-0.011068,1.300661,...,3.852705,1.732051,1.457738,2.031010,1.140465,0.000045,5.419034,1.865311,11.914470,1.0
5427,16292,13,13.043333,10.228055,8.770500e-08,178.72,26.086659,-0.025427,0.013349,1.248520,...,3.611556,1.414214,1.500000,1.936492,1.117372,0.000296,5.107510,3.198133,10.267138,1.0
5428,16293,14,11.273333,32.085103,2.797583e-08,146.76,26.592907,0.008712,0.018979,1.296583,...,3.357578,1.414214,1.274755,1.854050,1.138676,0.000167,5.156831,5.664371,10.762074,1.0


In [15]:
X, y = features.drop(columns=['Sample_ID']), df['Label']

In [16]:
X.shape

(10864, 620)

In [17]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np

forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5
)
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100,
   verbose=2
)

boruta.fit(X.iloc[:, :].values, y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	620
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	62
Rejected: 	558
Iteration: 	9 / 100
Confirmed: 	30
Tentative: 	32
Rejected: 	558
Iteration: 	10 / 100
Confirmed: 	30
Tentative: 	32
Rejected: 	558
Iteration: 	11 / 100
Confirmed: 	30
Tentative: 	32
Rejected: 	558
Iteration: 	12 / 100
Confirmed: 	32
Tentative: 	30
Rejected: 	558
Iteration: 	13 / 100
Confirmed: 	32
Tentative: 	30
Rejected: 	558
Iteration: 	14 / 100
Confirmed: 	32
Tentative: 	30
Rejected: 	558
Iteration: 	15 / 100
Confirmed: 	32
Tentative: 	30
Rejected: 	558
Iteration: 	16 / 100
Confirmed: 	3

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=219,
                                         n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x7FD641CB2C40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FD641CB2C40, verbose=2)

In [ ]:
boruta.support_

In [49]:
boruta.ranking_

array([276, 259, 215, 461, 162, 231, 168, 198,  45, 263, 334, 296, 188,
       291, 250, 461, 118, 279, 111, 287, 296, 275, 300,   1,  76,  68,
       271, 274,  51, 461, 245, 215,  93, 461,  28, 191, 204,  81, 214,
        60,  83, 108, 103, 151, 172, 256,  61,  23,   1, 170, 141,  39,
        22, 127, 116, 120, 130, 131,  95, 143, 147,  70,   1,   1, 143,
       119,   6, 209,  52,  57,   1, 141, 135,   2, 300, 129, 115,   1,
       180, 139, 110, 105,  81,  98, 112, 220, 102, 270, 190,  52,  13,
       242,  48, 208, 185, 261, 156, 236, 273, 253, 194,   1,  56, 202,
         1,   1,   1,  99, 133,   1, 287,  28, 117,   2, 318,  11,  17,
         1, 183,   1, 163, 244, 209, 114,  26,  48,   1, 185,   1, 188,
       248, 223, 137, 153,   2, 198,   6, 163, 251, 219, 133,  20, 140,
         1, 272, 188, 146, 157,  61, 201, 269, 206, 172,  36, 159,  50,
       234, 207,  71,   1, 242, 229, 127, 334, 106,   1, 212,  15, 193,
        93, 254, 220, 240, 232, 222, 264,  64,  86,  77, 213, 24

In [20]:
X_filtered = boruta.transform(X.values)

In [18]:
X.columns[boruta.support_]

Index(['ABSOLUTE(placurv)', 'aspect + slope_x_twi', 'elevation + slope_var',
       'elevation + slope_x_twi', 'elevation_bins + rel_sdoif',
       'elevation_bins + slope_x_twi', 'elevation_var + slope_var',
       'geology + placurv', 'geology + procurv', 'geology + rel_sdoif',
       'geology + sdoif', 'placurv + ring1_elevation', 'placurv + sdoif',
       'procurv + ring1_elevation', 'procurv + sdoif',
       'ring1_elevation + sdoif', 'sdoif + weathered', 'slope + slope_x_twi',
       'aspect * procurv', 'aspect * slope_var', 'aspect * slope_x_twi',
       'elevation * elevation_dif', 'elevation * geology', 'elevation * sdoif',
       'elevation * slope_x_twi', 'elevation_bins * slope_var',
       'elevation_bins * slope_x_twi', 'elevation_dif * sdoif',
       'elevation_var * geology', 'elevation_var * placurv',
       'elevation_var * sdoif', 'elevation_var * slope_var',
       'elevation_var * weathered', 'geology * sdoif', 'geology * slope_x_twi',
       'geology * weathered',

In [22]:
new_df = pd.DataFrame(X_filtered, columns=X.columns[boruta.support_])
new_df.head()

,ABSOLUTE(placurv),aspect + slope_x_twi,elevation + slope_var,elevation + slope_x_twi,elevation_bins + rel_sdoif,elevation_bins + slope_x_twi,elevation_var + slope_var,geology + placurv,geology + procurv,geology + rel_sdoif,...,placurv * slope_var,placurv * slope_x_twi,rel_sdoif * ring1_elevation,ring1_elevation * sdoif,ring1_elevation * slope_var,sdoif * slope_x_twi,sdoif * weathered,slope_var * weathered,NATURAL_LOGARITHM(sdoif_var),SQUARE_ROOT(sdoif_var)
0,0.012586,127.127109,141.932196,241.567109,1.992366,124.127109,56.082196,3.012586,2.996698,2.992366,...,0.283083,1.537072,-0.027672,4.646271,81.534212,156.533979,1.28173,22.492196,-19.99224,0.000046
1,0.010853,140.757992,175.487778,288.957992,3.05955,135.757992,36.037778,3.010853,2.994517,3.05955,...,0.209334,1.440845,0.156318,3.568895,50.630417,180.494979,1.359579,19.287778,-19.573874,0.000056
2,0.027318,184.5625,248.83414,335.1225,3.065026,175.5625,128.197474,2.027318,2.018734,2.065026,...,2.356837,4.714062,0.227591,4.777692,301.959491,235.557303,1.365055,86.27414,-19.525621,0.000058
3,0.031775,79.10367,89.751414,147.50367,0.820461,72.10367,19.934748,2.031775,2.009019,1.820461,...,0.424241,2.259318,-0.201982,1.238429,15.020341,78.272768,1.100826,13.351414,-18.201111,0.000112
4,0.005641,130.186959,116.486387,234.346959,1.994858,127.186959,22.966387,4.994359,4.991327,4.994858,...,-0.041326,-0.706149,-0.010926,2.728972,15.568573,160.767847,0.0,0.0,-17.23904,0.000181


In [26]:
new_df['Label'] = df['Label']
new_df['Sample_ID'] = df['Sample_ID']
new_df.head()

,ABSOLUTE(placurv),aspect + slope_x_twi,elevation + slope_var,elevation + slope_x_twi,elevation_bins + rel_sdoif,elevation_bins + slope_x_twi,elevation_var + slope_var,geology + placurv,geology + procurv,geology + rel_sdoif,...,rel_sdoif * ring1_elevation,ring1_elevation * sdoif,ring1_elevation * slope_var,sdoif * slope_x_twi,sdoif * weathered,slope_var * weathered,NATURAL_LOGARITHM(sdoif_var),SQUARE_ROOT(sdoif_var),Label,Sample_ID
0,0.012586,127.127109,141.932196,241.567109,1.992366,124.127109,56.082196,3.012586,2.996698,2.992366,...,-0.027672,4.646271,81.534212,156.533979,1.28173,22.492196,-19.99224,0.000046,0,1
1,0.010853,140.757992,175.487778,288.957992,3.05955,135.757992,36.037778,3.010853,2.994517,3.05955,...,0.156318,3.568895,50.630417,180.494979,1.359579,19.287778,-19.573874,0.000056,1,2
2,0.027318,184.5625,248.83414,335.1225,3.065026,175.5625,128.197474,2.027318,2.018734,2.065026,...,0.227591,4.777692,301.959491,235.557303,1.365055,86.27414,-19.525621,0.000058,0,3
3,0.031775,79.10367,89.751414,147.50367,0.820461,72.10367,19.934748,2.031775,2.009019,1.820461,...,-0.201982,1.238429,15.020341,78.272768,1.100826,13.351414,-18.201111,0.000112,0,4
4,0.005641,130.186959,116.486387,234.346959,1.994858,127.186959,22.966387,4.994359,4.991327,4.994858,...,-0.010926,2.728972,15.568573,160.767847,0.0,0.0,-17.23904,0.000181,0,5


In [27]:
new_df.to_csv('new_df.csv', index=False)

In [33]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#landslides = new_df[df['Label'] == 1]
#nonlandslides = new_df[df['Label'] == 0]

#less_nonslides = nonlandslides.sample(n=len(landslides))
#t = pd.concat([less_nonslides, landslides])

#maincols = df.columns.difference(['Sample_ID', 'Label'])

X = new_df.drop(columns=['Sample_ID', 'Label']) #t[maincols]
y = new_df['Label'] #t['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)

model = RandomForestClassifier(random_state=42, n_estimators=400)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Check the auc score of the model
print(f'RandomForest F1 score on the X_test is: {f1_score(y_test, y_pred)}\n')

# print classification report
# print(classification_report(y_test, y_pred))

RandomForest F1 score on the X_test is: 0.6179401993355483



NameError: name 'classification_report' is not defined

In [46]:
new_test = features.loc[:, ['ABSOLUTE(placurv)', 'aspect + slope_x_twi', 'elevation + slope_var',
       'elevation + slope_x_twi', 'elevation_bins + rel_sdoif',
       'elevation_bins + slope_x_twi', 'elevation_var + slope_var',
       'geology + placurv', 'geology + procurv', 'geology + rel_sdoif',
       'geology + sdoif', 'placurv + ring1_elevation', 'placurv + sdoif',
       'procurv + ring1_elevation', 'procurv + sdoif',
       'ring1_elevation + sdoif', 'sdoif + weathered', 'slope + slope_x_twi',
       'aspect * procurv', 'aspect * slope_var', 'aspect * slope_x_twi',
       'elevation * elevation_dif', 'elevation * geology', 'elevation * sdoif',
       'elevation * slope_x_twi', 'elevation_bins * slope_var',
       'elevation_bins * slope_x_twi', 'elevation_dif * sdoif',
       'elevation_var * geology', 'elevation_var * placurv',
       'elevation_var * sdoif', 'elevation_var * slope_var',
       'elevation_var * weathered', 'geology * sdoif', 'geology * slope_x_twi',
       'geology * weathered', 'placurv * slope_var', 'placurv * slope_x_twi',
       'rel_sdoif * ring1_elevation', 'ring1_elevation * sdoif',
       'ring1_elevation * slope_var', 'sdoif * slope_x_twi',
       'sdoif * weathered', 'slope_var * weathered',
       'NATURAL_LOGARITHM(sdoif_var)', 'SQUARE_ROOT(sdoif_var)']]
new_test.shape

(5430, 46)

In [47]:
new_test['Sample_ID'] = features['Sample_ID']
new_test.shape

(5430, 47)

In [48]:
new_test.to_csv('new_test.csv')